### Chapter 19: Advanced SystemVerilog Features

This chapter covers advanced SystemVerilog features that provide powerful capabilities for complex design and verification scenarios. These features enable efficient data manipulation, interface with external languages, and provide sophisticated control mechanisms.

#### Packed Unions

Packed unions allow multiple data types to share the same memory space, enabling efficient memory usage and type conversion.

##### Basic Packed Union Syntax

```systemverilog
typedef union packed {
    logic [31:0] word;
    logic [15:0] half_word[2];
    logic [7:0]  byte_data[4];
    struct packed {
        logic [7:0] a, b, c, d;
    } bytes;
} data_union_t;

module packed_union_example;
    data_union_t data;
    
    initial begin
        // Write as word
        data.word = 32'h12345678;
        $display("Word: %h", data.word);
        
        // Read as half words
        $display("Half words: %h %h", data.half_word[1], data.half_word[0]);
        
        // Read as bytes
        $display("Bytes: %h %h %h %h", 
                 data.byte_data[3], data.byte_data[2], 
                 data.byte_data[1], data.byte_data[0]);
        
        // Access struct members
        $display("Struct bytes: a=%h, b=%h, c=%h, d=%h",
                 data.bytes.a, data.bytes.b, data.bytes.c, data.bytes.d);
    end
endmodule
```

##### Packed Union with Different Data Types

```systemverilog
typedef union packed {
    logic [31:0] instruction;
    struct packed {
        logic [5:0]  opcode;
        logic [4:0]  rs;
        logic [4:0]  rt;
        logic [4:0]  rd;
        logic [4:0]  shamt;
        logic [5:0]  funct;
    } r_type;
    struct packed {
        logic [5:0]  opcode;
        logic [4:0]  rs;
        logic [4:0]  rt;
        logic [15:0] immediate;
    } i_type;
    struct packed {
        logic [5:0]  opcode;
        logic [25:0] address;
    } j_type;
} instruction_t;

module instruction_decoder;
    instruction_t instr;
    
    task decode_instruction(input logic [31:0] raw_instr);
        instr.instruction = raw_instr;
        
        case (instr.r_type.opcode)
            6'b000000: begin // R-type
                $display("R-type: rs=%d, rt=%d, rd=%d, funct=%b",
                        instr.r_type.rs, instr.r_type.rt, 
                        instr.r_type.rd, instr.r_type.funct);
            end
            6'b001000: begin // I-type (ADDI)
                $display("I-type: rs=%d, rt=%d, imm=%d",
                        instr.i_type.rs, instr.i_type.rt, 
                        $signed(instr.i_type.immediate));
            end
            6'b000010: begin // J-type (JUMP)
                $display("J-type: address=%h", instr.j_type.address);
            end
        endcase
    endtask
    
    initial begin
        decode_instruction(32'h00851020); // ADD R2, R4, R5
        decode_instruction(32'h20420005); // ADDI R2, R2, 5
        decode_instruction(32'h08000100); // J 0x400
    end
endmodule
```

#### Tagged Unions

Tagged unions provide type safety by maintaining information about which member is currently active.

##### Basic Tagged Union

```systemverilog
typedef union tagged {
    void Invalid;
    int Integer;
    real Real;
    string String;
} value_t;

module tagged_union_example;
    value_t values[4];
    
    initial begin
        // Initialize tagged union members
        values[0] = tagged Invalid;
        values[1] = tagged Integer 42;
        values[2] = tagged Real 3.14159;
        values[3] = tagged String "Hello World";
        
        // Process each value
        foreach (values[i]) begin
            $display("Value[%0d]: ", i);
            case (values[i]) matches
                tagged Invalid: 
                    $display("  Invalid value");
                tagged Integer .n: 
                    $display("  Integer: %0d", n);
                tagged Real .r: 
                    $display("  Real: %f", r);
                tagged String .s: 
                    $display("  String: %s", s);
            endcase
        end
    end
endmodule
```

##### Complex Tagged Union Example

```systemverilog
typedef union tagged {
    void Empty;
    struct {
        int x, y;
    } Point;
    struct {
        int x, y, width, height;
    } Rectangle;
    struct {
        int x, y, radius;
    } Circle;
} shape_t;

module shape_processor;
    shape_t shapes[$];
    
    function real calculate_area(shape_t shape);
        case (shape) matches
            tagged Empty: return 0.0;
            tagged Point: return 0.0;
            tagged Rectangle .r: 
                return real'(r.width * r.height);
            tagged Circle .c: 
                return 3.14159 * real'(c.radius * c.radius);
        endcase
    endfunction
    
    initial begin
        // Create different shapes
        shapes.push_back(tagged Point '{10, 20});
        shapes.push_back(tagged Rectangle '{0, 0, 50, 30});
        shapes.push_back(tagged Circle '{25, 25, 10});
        
        foreach (shapes[i]) begin
            $display("Shape %0d area: %f", i, calculate_area(shapes[i]));
        end
    end
endmodule
```

#### Streaming Operators

Streaming operators provide efficient ways to pack and unpack data structures.

##### Pack and Unpack Operations

```systemverilog
module streaming_operators_example;
    typedef struct packed {
        logic [7:0] header;
        logic [15:0] data;
        logic [7:0] checksum;
    } packet_t;
    
    packet_t packet;
    logic [31:0] packed_data;
    logic [7:0] byte_array[4];
    
    initial begin
        // Initialize packet
        packet.header = 8'hAA;
        packet.data = 16'h1234;
        packet.checksum = 8'h55;
        
        // Pack using streaming operator
        packed_data = {>>{packet}};
        $display("Packed data: %h", packed_data);
        
        // Unpack using streaming operator
        {>>{packet}} = packed_data;
        $display("Unpacked - Header: %h, Data: %h, Checksum: %h",
                 packet.header, packet.data, packet.checksum);
        
        // Pack into byte array
        {>>{byte_array}} = packet;
        $display("Byte array: %h %h %h %h", 
                 byte_array[0], byte_array[1], byte_array[2], byte_array[3]);
    end
endmodule
```

##### Streaming with Different Sizes

```systemverilog
module streaming_sizes_example;
    logic [63:0] data64;
    logic [31:0] data32[2];
    logic [15:0] data16[4];
    logic [7:0]  data8[8];
    
    initial begin
        data64 = 64'h123456789ABCDEF0;
        
        // Stream to different sizes
        {>>{data32}} = data64;
        {>>{data16}} = data64;
        {>>{data8}}  = data64;
        
        $display("Original: %h", data64);
        $display("32-bit chunks: %h %h", data32[0], data32[1]);
        $display("16-bit chunks: %h %h %h %h", 
                 data16[0], data16[1], data16[2], data16[3]);
        $display("8-bit chunks: %h %h %h %h %h %h %h %h",
                 data8[0], data8[1], data8[2], data8[3],
                 data8[4], data8[5], data8[6], data8[7]);
        
        // Stream back
        logic [63:0] reconstructed;
        {>>{reconstructed}} = data8;
        $display("Reconstructed: %h", reconstructed);
    end
endmodule
```

#### DPI (Direct Programming Interface)

DPI allows SystemVerilog to interface with C/C++ functions, enabling powerful mixed-language simulation.

##### Basic DPI Import

```systemverilog
// DPI function declarations
import "DPI-C" function int c_add(input int a, input int b);
import "DPI-C" function void c_print_message(input string msg);
import "DPI-C" function real c_sqrt(input real x);

module dpi_example;
    initial begin
        int result;
        real sqrt_result;
        
        // Call C functions
        result = c_add(10, 20);
        $display("C add result: %0d", result);
        
        c_print_message("Hello from SystemVerilog!");
        
        sqrt_result = c_sqrt(16.0);
        $display("Square root of 16: %f", sqrt_result);
    end
endmodule
```

##### DPI Export (SystemVerilog to C)

```systemverilog
// Export SystemVerilog functions to C
export "DPI-C" function sv_callback;
export "DPI-C" task sv_display_data;

// Import C function that will call back
import "DPI-C" function void c_process_with_callback();

function int sv_callback(input int value);
    $display("SV callback called with: %0d", value);
    return value * 2;
endfunction

task sv_display_data(input int data[], input int size);
    $display("Displaying %0d elements:", size);
    for (int i = 0; i < size; i++) begin
        $display("  [%0d] = %0d", i, data[i]);
    end
endtask

module dpi_export_example;
    initial begin
        c_process_with_callback();
    end
endmodule
```

##### DPI with Complex Data Types

```systemverilog
typedef struct {
    int id;
    real value;
    string name;
} record_t;

// DPI functions with complex types
import "DPI-C" function void c_process_record(input record_t rec);
import "DPI-C" function record_t c_create_record(input int id);

module dpi_complex_example;
    record_t my_record;
    
    initial begin
        // Create record in C
        my_record = c_create_record(123);
        $display("Created record: id=%0d, value=%f, name=%s",
                 my_record.id, my_record.value, my_record.name);
        
        // Modify and send to C
        my_record.value = 42.5;
        my_record.name = "Modified";
        c_process_record(my_record);
    end
endmodule
```

#### System Tasks and Functions

SystemVerilog provides numerous built-in system tasks and functions for various purposes.

##### File I/O System Tasks

```systemverilog
module file_io_example;
    int file_handle;
    string filename = "output.txt";
    int data_array[10] = '{0,1,2,3,4,5,6,7,8,9};
    
    initial begin
        // Open file for writing
        file_handle = $fopen(filename, "w");
        if (file_handle == 0) begin
            $error("Failed to open file: %s", filename);
            $finish;
        end
        
        // Write data to file
        $fwrite(file_handle, "Data Array Contents:\n");
        foreach (data_array[i]) begin
            $fwrite(file_handle, "data[%0d] = %0d\n", i, data_array[i]);
        end
        
        // Close file
        $fclose(file_handle);
        
        // Read file back
        file_handle = $fopen(filename, "r");
        if (file_handle != 0) begin
            string line;
            $display("File contents:");
            while (!$feof(file_handle)) begin
                $fgets(line, file_handle);
                $write("%s", line);
            end
            $fclose(file_handle);
        end
    end
endmodule
```

##### Memory and String System Functions

```systemverilog
module system_functions_example;
    string str1 = "Hello";
    string str2 = "World";
    string result;
    
    int memory[1024];
    
    initial begin
        // String functions
        result = {str1, " ", str2};
        $display("Concatenated: %s", result);
        $display("Length of '%s': %0d", result, result.len());
        $display("Substring: %s", result.substr(0, 4));
        
        // Case conversion
        $display("Upper case: %s", result.toupper());
        $display("Lower case: %s", result.tolower());
        
        // Memory functions
        $display("Memory size: %0d bytes", $bits(memory));
        $display("Array size: %0d elements", $size(memory));
        
        // Random number generation
        $display("Random numbers:");
        for (int i = 0; i < 5; i++) begin
            $display("  %0d", $urandom_range(1, 100));
        end
    end
endmodule
```

##### Assertion System Tasks

```systemverilog
module assertion_system_tasks;
    logic clk = 0;
    logic reset = 1;
    logic [7:0] counter = 0;
    
    always #5 clk = ~clk;
    
    initial begin
        #20 reset = 0;
        #200 $finish;
    end
    
    always @(posedge clk) begin
        if (reset) begin
            counter <= 0;
        end else begin
            counter <= counter + 1;
            
            // Assertion system tasks
            if (counter < 100) begin
                $info("Counter value: %0d", counter);
            end else if (counter < 200) begin
                $warning("Counter getting high: %0d", counter);
            end else begin
                $error("Counter overflow: %0d", counter);
                $fatal("Simulation terminated due to counter overflow");
            end
            
            // Immediate assertions
            assert (counter <= 255) else 
                $error("Counter exceeded maximum value");
        end
    end
endmodule
```

#### Compiler Directives

Compiler directives control compilation behavior and provide conditional compilation capabilities.

##### Conditional Compilation

```systemverilog
`define DEBUG_MODE
`define SYNTHESIS_MODE

`ifdef DEBUG_MODE
    `define DBG_PRINT(msg) $display("[DEBUG] %s", msg)
`else
    `define DBG_PRINT(msg)
`endif

module compiler_directives_example;
    logic [31:0] data;
    
    initial begin
        data = 32'h12345678;
        
        `DBG_PRINT("Starting simulation")
        
        `ifdef SYNTHESIS_MODE
            $display("Synthesis mode enabled");
        `else
            $display("Simulation mode");
        `endif
        
        `ifndef FAST_SIM
            #100; // Add delay only if FAST_SIM not defined
        `endif
        
        `DBG_PRINT("Simulation complete")
    end
endmodule
```

##### Macro Definitions

```systemverilog
// Function-like macros
`define MAX(a,b) ((a) > (b) ? (a) : (b))
`define MIN(a,b) ((a) < (b) ? (a) : (b))
`define CLAMP(val,min,max) `MIN(`MAX(val,min),max)

// Multi-line macros
`define ASSERT_CLK(clk, cond, msg) \
    always @(posedge clk) begin \
        assert(cond) else $error(msg); \
    end

// Parameterized macros
`define REG_ARRAY(name, width, depth) \
    logic [width-1:0] name [depth-1:0]

module macro_example;
    `REG_ARRAY(memory, 32, 1024);
    
    logic clk = 0;
    logic [7:0] value = 0;
    
    always #5 clk = ~clk;
    
    `ASSERT_CLK(clk, value < 200, "Value exceeded limit")
    
    initial begin
        int a = 10, b = 20, c = 15;
        
        $display("MAX(%0d, %0d) = %0d", a, b, `MAX(a, b));
        $display("MIN(%0d, %0d) = %0d", a, b, `MIN(a, b));
        $display("CLAMP(%0d, %0d, %0d) = %0d", 25, a, b, `CLAMP(25, a, b));
        
        repeat (10) @(posedge clk) value++;
        
        $finish;
    end
endmodule
```

##### Include and Timescale Directives

```systemverilog
// File: common_defines.sv
`ifndef COMMON_DEFINES_SV
`define COMMON_DEFINES_SV

`define ADDR_WIDTH 32
`define DATA_WIDTH 64
`define CACHE_SIZE 1024

typedef logic [`ADDR_WIDTH-1:0] addr_t;
typedef logic [`DATA_WIDTH-1:0] data_t;

`endif // COMMON_DEFINES_SV

// File: main_module.sv
`timescale 1ns/1ps

`include "common_defines.sv"

module main_module;
    addr_t address;
    data_t data;
    
    initial begin
        address = `ADDR_WIDTH'h12345678;
        data = `DATA_WIDTH'h123456789ABCDEF0;
        
        $display("Address width: %0d bits", `ADDR_WIDTH);
        $display("Data width: %0d bits", `DATA_WIDTH);
        $display("Cache size: %0d entries", `CACHE_SIZE);
        
        #1.5 $display("Time: %t", $time);
        $finish;
    end
endmodule
```

##### Pragma Directives

```systemverilog
module pragma_example;
    // Synthesis pragmas
    (* synthesis, keep *) logic important_signal;
    (* synthesis, dont_touch *) logic critical_path;
    
    // Simulation pragmas
    (* simulator, translate_off *)
    initial begin
        $display("This code only runs in simulation");
    end
    (* simulator, translate_on *)
    
    // Coverage pragmas
    logic [3:0] state;
    
    always @* begin
        case (state)
            4'b0000: important_signal = 1'b0;
            4'b0001: important_signal = 1'b1;
            // synthesis translate_off
            default: $error("Invalid state: %b", state);
            // synthesis translate_on
        endcase
    end
    
    // Lint pragmas
    //synopsys translate_off
    always @(important_signal) begin
        if (important_signal)
            $display("Important signal asserted");
    end
    //synopsys translate_on
    
endmodule
```

#### Summary

This chapter covered advanced SystemVerilog features that provide powerful capabilities for complex designs:

- **Packed Unions**: Enable efficient memory usage and type conversion by allowing multiple data types to share memory space
- **Tagged Unions**: Provide type-safe unions with runtime type information
- **Streaming Operators**: Offer efficient data packing and unpacking capabilities
- **DPI**: Enables seamless integration with C/C++ code for enhanced functionality
- **System Tasks and Functions**: Provide built-in capabilities for file I/O, debugging, and simulation control
- **Compiler Directives**: Enable conditional compilation, macro definition, and compilation control

These advanced features are essential for creating sophisticated verification environments, implementing complex data structures, and interfacing with external tools and languages. They enable SystemVerilog to be used effectively in large-scale, professional verification and design projects.